In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
import os
import numpy as np
import random
import librosa
import h5py
import tensorflow as tf
import keras
from keras.models import Model
from keras.layers import Input, Conv2D, BatchNormalization, MaxPooling2D, Flatten, Activation, Lambda
import keras.regularizers as regularizers
from keras.optimizers import Adam
from l3embedding.audio import pcm2float
from resampy import resample
import pescador
from skimage import img_as_float

Using TensorFlow backend.


In [3]:
def shuffle_files(iterable):
    lst = list(iterable)
    random.shuffle(lst)
    return iter(lst)

In [4]:
def quant_data_generator(data_dir, batch_size=512, random_state=None, start_batch_idx=None):
    if random_state:
        random.seed(random_state)

    batch = None
    curr_batch_size = 0
    batch_idx = 0
        
    for fname in shuffle_files(os.listdir(data_dir)):
        print(fname)
        data_batch_path = os.path.join(data_dir, fname)
        #shortlist_files.append(data_batch_path)
        blob_start_idx = 0

        data_blob = np.load(data_batch_path)
        blob_size = len(data_blob['db_mels'])

        while blob_start_idx < blob_size:
            blob_end_idx = min(blob_start_idx + batch_size - curr_batch_size, blob_size)

            # If we are starting from a particular batch, skip computing all of
            # the prior batches
            if start_batch_idx is None or batch_idx >= start_batch_idx:
                if batch is None:
                    batch = data_blob['db_mels'][blob_start_idx:blob_end_idx]
                else:
                    batch = np.concatenate([batch, data_blob['db_mels'][blob_start_idx:blob_end_idx]])

            curr_batch_size += blob_end_idx - blob_start_idx
            blob_start_idx = blob_end_idx

            if blob_end_idx == blob_size:
                data_blob.close()

            if curr_batch_size == batch_size:
                X = []
                # If we are starting from a particular batch, skip yielding all
                # of the prior batches
                if start_batch_idx is None or batch_idx >= start_batch_idx:
                    #saved audio files are already in float so need not convert to float32
                    X = [batch[i] for i in range(batch_size)]

                    batch = np.array(X)[:, :, :, np.newaxis]
                    #print(np.shape(batch)) #(64, 256, 191, 1)
                    return batch

                batch_idx += 1
                curr_batch_size = 0
                batch = None

def single_epoch_test_data_generator(file_list, batch_size=64, start_batch_idx=None):
    batch = None
    curr_batch_size = 0
    batch_idx = 0

    for fname in file_list:
        data_batch_path = fname
        blob_start_idx = 0

        data_blob = np.load(data_batch_path)
        blob_size = len(data_blob['db_mels'])

        while blob_start_idx < blob_size:
            blob_end_idx = min(blob_start_idx + batch_size - curr_batch_size, blob_size)

            # If we are starting from a particular batch, skip computing all of
            # the prior batches
            if start_batch_idx is None or batch_idx >= start_batch_idx:
                if batch is None:
                    batch = data_blob['db_mels'][blob_start_idx:blob_end_idx]
                else:
                    batch = np.concatenate([batch, data_blob['db_mels'][blob_start_idx:blob_end_idx]])

            curr_batch_size += blob_end_idx - blob_start_idx
            blob_start_idx = blob_end_idx

            if blob_end_idx == blob_size:
                data_blob.close()

            if curr_batch_size == batch_size:
                X = []
                if start_batch_idx is None or batch_idx >= start_batch_idx:
                    X = [batch[i] for i in range(batch_size)]
                    batch = np.array(X)[:, :, :, np.newaxis]
                    #print(np.shape(batch)) #(64, 256, 191, 1)
                    yield batch

                batch_idx += 1
                curr_batch_size = 0
                batch = None

In [5]:
def quantize_keras_to_tflite(tflite_model_file, keras_model, keras_model_path, quant_mode='default', quantized_input=True,\
                            halved_convs=False, calibrate_data_dir=None, num_calibration_steps=1024):

    def representative_dataset_gen():
            print('Calibrating.........')
            for _ in range(num_calibration_steps):
                x = quant_data_generator(calibrate_data_dir, batch_size=1)
                yield [np.array(x).astype(np.float32)]
    
    tf_version = tf.__version__.split('.')[0]
    
    if tf_version == '2':
        converter = tf.lite.TFLiteConverter.from_keras_model(keras_model)
    else:
        converter = tf.lite.TFLiteConverter.from_keras_model_file(keras_model_path)
    
    if quant_mode == 'default':
        if calibrate_data_dir is None:
            raise ValueError('Quantized activation calibration needs data directory!')
        
        converter.optimizations = [tf.lite.Optimize.DEFAULT]
        print('quantized_input={}'.format(quantized_input))
        if quantized_input:
            #converter.inference_input_type = tf.int8
            converter.inference_output_type = tf.int8
        #converter.default_ranges_stats = (-127, 128)
        converter.representative_dataset = representative_dataset_gen
                
    elif quant_mode == 'size':
        converter.post_training_quantize = True
        converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
    else:
        raise ValueError('Unrecognized Quantization mode!')

    tflite_model = converter.convert()
    with open(tflite_model_file, "wb") as f:
        f.write(tflite_model)
    print('Tflite model saved in:', tflite_model_file)

In [6]:
def post_training_quantization(model_path, calibrate_data_dir, quant_mode='default', quantized_input=True, \
                               halved_convs=False, flatten=False, calibration_steps=1024):
    
    #1. Convert l3model to keras model for quantization (with maxpooling layer but flatten removed)
    dir_prefix = '/scratch/sk7898/quantization/' + os.path.basename(model_path).strip('.h5')
    
    if not os.path.isdir(dir_prefix):
        os.makedirs(dir_prefix)
    
    tf_version = tf.__version__.split('.')[0]
    
    if tf_version=='2':
        keras_model = tf.keras.models.load_model(model_path)
    else:
        keras_model = keras.models.load_model(model_path)
    #print(keras_model.summary())
    
    #2.1 Convert keras to tflite model
    #2.2 Quantize model with mode 'default' for only weights quantization or 'size' for full quantization
    #2.3 Save the quantized tflite model
    
    print('Quantizing keras model and saving as tflite')
    input_type = '_float32'
    tflite_model_file = os.path.join(dir_prefix, 'tf_' + str(tf_version) + '_full_quantized_'+ quant_mode + input_type + '.tflite')
    
    quantize_keras_to_tflite(tflite_model_file, keras_model, model_path, quant_mode=quant_mode,\
                             quantized_input=quantized_input, halved_convs=halved_convs, \
                             calibrate_data_dir=calibrate_data_dir, num_calibration_steps=calibration_steps)

**Quantize both the weights and the activations of the model**\
If the input is already in int8, set quantized_input = True\
If tflite should convert the float32 to int8 by adding a Quantize layer, quantized_input = False

In [7]:
#model_path = '/scratch/sk7898/l3pruning/embedding/fixed/reduced_input/l3_audio_original_48000_256_242_2048.h5'
#model_path = '/scratch/dr2915/l3pruning/embedding/fixed/reduced_input/l3_audio_20191108201753_8000_64_160_1024_half.h5'
model_path = '/scratch/dr2915/Nathan/pipeline_cmsis_mels.h5'
calibrate_data_dir = '/scratch/sk7898/cmsis_ml_data' #'/beegfs/dr2915/sonyc_ust/frames/8KHz'
calibration_steps = 32

quant_mode='default'
flatten=True
quantized_input=True
halved_convs=True if 'half' in model_path else False

post_training_quantization(model_path, calibrate_data_dir, quant_mode=quant_mode, quantized_input=quantized_input,\
                           halved_convs=halved_convs, flatten=flatten, calibration_steps=calibration_steps)

Quantizing keras model and saving as tflite
quantized_input=True
Calibrating.........
04_002755.npz
04_002755.npz
01_001297.npz
03_000965.npz
01_001297.npz
01_001297.npz
03_000965.npz
03_000965.npz
03_000965.npz
01_001297.npz
04_002755.npz
01_001297.npz
03_000965.npz
01_001297.npz
03_000965.npz
01_001297.npz
04_002755.npz
03_000965.npz
04_002755.npz
01_001297.npz
03_000965.npz
01_001297.npz
01_001297.npz
04_002755.npz
01_001297.npz
04_002755.npz
04_002755.npz
03_000965.npz
01_001297.npz
01_001297.npz
03_000965.npz
01_001297.npz
Tflite model saved in: /scratch/sk7898/quantization/pipeline_cmsis_mels/tf_2_full_quantized_default_float32.tflite


**Input/Output of tflite model (Interpreter)**

In [8]:
output_path = '/scratch/sk7898/quantization'
quant_model = 'pipeline_cmsis_mels/tf_2_full_quantized_default_float32.tflite'
quant_output_path = os.path.join(output_path, quant_model)

interpreter = tf.lite.Interpreter(model_path=str(quant_output_path))
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

input_shape = input_details[0]['shape'][1:]
output_shape = output_details[0]['shape'][1:]
input_index = input_details[0]['index']
output_index = output_details[0]['index']

interpreter.allocate_tensors()

print("== Input details ==")
print(interpreter.get_input_details()[0])
print("type:", input_details[0]['dtype'])
print("\n== Output details ==")
print(interpreter.get_output_details()[0])

== Input details ==
{'name': 'input_1', 'index': 45, 'shape': array([ 1, 64, 51,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
type: <class 'numpy.float32'>

== Output details ==
{'name': 'Identity', 'index': 46, 'shape': array([1, 8], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}


**Generate Embedding from the tflite model**

In [9]:
def get_softmax_batch_from_tflite(data_gen, tflite_model_file, batch_size, classes=8):
    
    predictions = []
    interpreter = tf.lite.Interpreter(model_path=str(tflite_model_file))
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    input_shape = input_details[0]['shape'][1:]
    output_shape = output_details[0]['shape'][1:]
    input_index = input_details[0]['index']
    output_index = output_details[0]['index']

    interpreter.resize_tensor_input(input_index, ((batch_size, ) + tuple(input_shape)))
    interpreter.resize_tensor_input(output_index, ((batch_size, ) + tuple(input_shape)))
    interpreter.allocate_tensors()
    
    print("== Input details ==")
    print(interpreter.get_input_details()[0])
    print("type:", input_details[0]['dtype'])
    print("\n== Output details ==")
    print(interpreter.get_output_details()[0])
       
    #predictions per batch   
    for idx, batch_x in enumerate(data_gen):
        x = np.array(batch_x).astype(np.float32)
        interpreter.set_tensor(input_index, x)
        interpreter.invoke()
        output = interpreter.get_tensor(output_index)
        predictions.append(output)
        
    return predictions

In [10]:
def gen_softmax(tflite_model_file, file_list, batch_size=64):
    
    output = None
    classes = 8
    print('Getting softmax output for downstream classes out of Quantized tflite model')
    
    data_gen = single_epoch_test_data_generator(file_list, batch_size=batch_size)

    output = get_softmax_batch_from_tflite(data_gen, tflite_model_file, batch_size, classes=classes)
    return output

In [11]:
def get_test_files(data_dir, num_files=10):
    shortlist_files = []
    random.seed(23455)
    
    for fname in shuffle_files(os.listdir(data_dir)):
        data_batch_path = os.path.join(data_dir, fname)
        shortlist_files.append(data_batch_path)
        if len(shortlist_files) >= num_files:
            break
    return shortlist_files

In [12]:
out_file = 'selected_audio_files_cmsis_mel.npz'
tflite_model_file = '/scratch/sk7898/quantization/pipeline_cmsis_mels/tf_2_full_quantized_default_float32.tflite'

test_data_dir = '/beegfs/dr2915/sonyc_ust/db_mels/test'
shortlist_files = get_test_files(test_data_dir)
np.savez(out_file, x=shortlist_files)

In [13]:
files = np.load(out_file)
shortlist_files = files['x']
print(shortlist_files)

['/beegfs/dr2915/sonyc_ust/db_mels/test/20_010340.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/00_010346.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/20_010593.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/06_010465.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/27_010536.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/10_010403.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/16_010802.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/34_010422.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/20_010651.npz'
 '/beegfs/dr2915/sonyc_ust/db_mels/test/16_010377.npz']


In [14]:
output = gen_softmax(tflite_model_file, shortlist_files, batch_size=91)

Getting softmax output for downstream classes out of Quantized tflite model
== Input details ==
{'name': 'input_1', 'index': 45, 'shape': array([91, 64, 51,  1], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}
type: <class 'numpy.float32'>

== Output details ==
{'name': 'Identity', 'index': 46, 'shape': array([91,  8], dtype=int32), 'dtype': <class 'numpy.float32'>, 'quantization': (0.0, 0)}


In [15]:
pred = np.array(output).reshape(-1, 8)
pred_max = np.argmax(pred, axis=-1) 
print(pred_max)

[1 1 1 1 1 0 0 1 1 0 1 1 1 0 1 0 1 0 1 1 3 1 1 1 1 0 0 0 0 0 0 1 0 0 1 1 1
 1 1 1 1 1 1 1 0 0 1 0 3 3 3 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 0 1 0 1 1
 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 0 3 0 3 0 0 0 0 0 0 3 0 0 0 3 0 3 3 3 3 3
 3 3 3 3 3 3 0 0 0 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 6 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 1 1 1 1 1 4 1 4 4 1 1 0 1 1 1 1 1 1 1 1 0 0 0 4 0 0 0 0 0 0 0 0 0 0 1 1
 1 0 1 1 1 1 1 1 1 0 0 0 0 0 1 1 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 2 6 6 2 6 6 6 6 6 6 6 6 6 6 2 6 6 6 6 6 5 6 6 6 6 6 6 6 6 6 6 6 6 5 2
 5 1 1 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 4 4 4 4 4 4 4 6 6 6 6 6 0
 0 0 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 2 2 1 2 2 1 6 6 6 6 6 6 6 6 0 6 0 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 6 0 4 4 4 4 4 4 4 4 4 